### 1.Preprocessing

In [806]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()
len(application_df)

34299

In [807]:
def rebin(col,bin_ratio):
    val_cnt = application_df[col].value_counts()
    # gather insigificant categories with value counts at least 100x fewer than the max
    rebins = val_cnt[val_cnt.values < max(val_cnt.values)/bin_ratio].index
    # Replace in dataframe
    for app in rebins:
        application_df[col] = application_df[col].replace(app,"Other")

In [808]:

print(application_df.nunique())

# if column values are boolean, mark for drop if ratio of values is greater than ratio
drop_ratio = 50
bin_ratio = 50
# if columns are non-boolean and object type, determine if categories can be binned
drop_list = []
bin_list = []
inspect_list = []
for col in application_df.columns:
    val_cnt = application_df[col].value_counts().reset_index(drop=True)

# drop if boolean and ratio of largest to smallest is >= 100 (lopsided to be predictive)
    if len(val_cnt) == 2:
        # val_cnt=val_cnt.reset_index(drop=True)
        if val_cnt[0]/val_cnt[1] >= drop_ratio:
            drop_list.append(col)

# drop if number of categories exceeds 1/3 number of columns in df (most likely ID-related info)
    elif application_df[col].nunique() > 0.33*len(application_df):
        drop_list.append(col)

# non-boolean object types not caught by the above filter will be gathered, filtered, and rebinned
    elif len(val_cnt) > 2:
        if application_df[col].dtype.kind == 'O':
            bin_list.append(col)
            rebin(col,bin_ratio)
        else:
            inspect_list.append(col)

# drops anything with value count < 2 for good measure
    else:
        drop_list.append(col)


application_df = application_df.drop(columns=drop_list, axis=1)
print(f"\nDROPPED: {drop_list}\n")
print(f"RE-BINNED: {bin_list}\n")
print(f"INSPECT: {inspect_list}\n")
print(application_df.nunique())


EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

DROPPED: ['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS']

RE-BINNED: ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

INSPECT: ['ASK_AMT']

APPLICATION_TYPE       8
AFFILIATION            3
CLASSIFICATION         7
USE_CASE               3
ORGANIZATION           4
INCOME_AMT             7
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [809]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
print(application_cat)

['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']


In [810]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/Users/max/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Independent,...,ORGANIZATION_Co-operative,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [811]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

/Users/max/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Co-operative,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [812]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values
print(y)
print(X)

# randomly select c

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

[1 1 0 ... 0 1 0]
[[5.0000000e+03 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0859000e+05 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [5.0000000e+03 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [5.0000000e+03 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [5.0000000e+03 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [3.6500179e+07 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 0.0000000e+00]]


/Users/max/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [813]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

len(X_train_scaled)

25724

### 2. Optimization

In [814]:
print(len(application_df.columns))

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

     # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=37, activation=activation))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=16,
            max_value=64,
            step=8),
            activation=activation))
        
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
    
    return nn_model

34


In [ ]:
# Import the kerastuner library
!pip install -q -U keras-tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=10,
    directory='checkpoints',
    hyperband_iterations=1,
    factor=5,
    project_name='Hyperband')

In [816]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100, validation_data=(X_test_scaled,y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.7351603507995605

Best val_accuracy So Far: 0.7369096279144287
Total elapsed time: 00h 00m 15s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation
4                 |1                 |num_layers
56                |32                |units_0
48                |None              |units_1
24                |None              |units_2
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0].values
best_hyper

{'activation': 'relu',
 'units_1': 24,
 'units_2': 48,
 'units_3': 40,
 'units_4': 64,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0005'}

In [ ]:
# Evaluate best model against full test data
# model = tuner.hypermodel.build(best_hyper)
# history = model.fit(X_train_scaled,y_train,epochs=50, validation_data=(X_test_scaled,y_test))
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5406 - accuracy: 0.7371 - 279ms/epoch - 1ms/step
Loss: 0.5406209230422974, Accuracy: 0.7371428608894348
